# Step-by-step Guidance for CIKM 2022 AnalytiCup Competition

## Step 1. 安装FederatedScope

In [ ]:
# 拷贝FederatedScope并切换到比赛所使用的分支cikm22competition

!cp -r fs_latest FederatedScope

# 如果不在playground中请输入命令`cd FederatedScope`
import os
os.chdir('FederatedScope')

!git checkout cikm22competition

## Step 2. 搭建运行环境

In [ ]:
# 安装 FS
!pip install -e . --user

## Step 3. 下载比赛数据集

In [ ]:
# 比赛数据集已经提前下载到`data`目录下，通过如下的命令进行解压:
!mkdir -p data
!cp -r ../data/CIKM22Competition ./data/

现在比赛数据集已经被放置在`FederatedScope/data/CIKM22Competition`目录下，遵循`CIKM22Competition/${client_id}`的方式进行组织，其中`${client_id}`是client的序号并从1开始计数。每个client的目录下包含训练（train.pt），测试（test.pt）和验证（val.pt）三个部分的数据。可以通过`torch.load`的方式来查看数据：

In [ ]:
import torch
# 加载序号为1的client的训练数据
train_data_client1 = torch.load('./data/CIKM22Competition/1/train.pt')
# 查看训练数据中的第一个训练样例
print(train_data_client1[0])
# 查看训练样例的label
print(train_data_client1[0].y)
# 查看训练样例的序号，即${sample_id}
print(train_data_client1[0].data_index)

## Step 4. 在比赛数据上运行baseline算法

FederatedScope为比赛数据内置了两个baseline算法："isolated training"和"FedAvg"。假设你已经成功的搭建好FederatedScope的运行环境，并下载了比赛数据，那么可以通过以下的命令运行两个baseline算法

In [ ]:
# 作为示例，我们在这里仅运行3轮，用户可以按照自己的需求更改运行轮数
# 按照如下命令运行isolated training
!python federatedscope/main.py --cfg federatedscope/gfl/baseline/isolated_gin_minibatch_on_cikmcup.yaml --client_cfg federatedscope/gfl/baseline/isolated_gin_minibatch_on_cikmcup_per_client.yaml federate.total_round_num 3

# 按照如下命令运行FedAvg
!python federatedscope/main.py --cfg federatedscope/gfl/baseline/fedavg_gin_minibatch_on_cikmcup.yaml --client_cfg federatedscope/gfl/baseline/fedavg_gin_minibatch_on_cikmcup_per_client.yaml federate.total_round_num 3

其中参数--cfg xxxx.yaml指定了全局的参数，--client_cfg xxx.yaml为每一个client单独指定了参数

## Step 5. 保存并提交预测结果

### 提交格式
如CIKM 2022 AnalytiCup Competition介绍中所述，参赛者需要将所有client的预测结果保存到一个csv文件中进行提交。在这个csv文件中，每一行代表一个测试样例的预测结果，并且由`${client_id}`和`${sample_id}`标识所预测的测试样例。其中，`${client_id}`从1开始计数，`${sample_id}`需要与比赛数据中的测试样例的序号保持一致（可以通过测试样例的`data_index`属性获得测试样例的序号）。

需要注意的是，分类任务和多维度回归任务分别遵循以下不同的格式：
* 对分类任务，每一行应当遵循如下的格式(`${category_id}`从0开始计数)：

```
${client_id},${sample_id},${category_id}
```

* 对于N维的回归任务，每一行应当遵循如下的格式：
```
${client_id},${sample_id},${prediction_1st_dimension},…,${prediction_N-th_dimension}
```

### 通过FederatedScope保存预测结果
FederatedScope中的"cikm22competition"分支目前支持在训练结束时保存预测结果（相关代码见`federatedscope/gfl/trainer/graphtrainer.py`和`federatedscope/core/trainers/torch_trainer.py`）。
预测结果将会被保存在一个名为prediction.csv的文件中，该文件位于`outdir`参数所指定的目录中。每次运行时，如果`outdir`所指定的目录已经存在，则会使用一个时间戳后缀用以区分。
在训练结束时，FederatedScope的训练记录也将标识出预测结果所在的目录，如下所示
![Finish saving prediction results](https://img.alicdn.com/imgextra/i1/O1CN01L68gve1nGu3BNp5vL_!!6000000005063-2-tps-4766-404.png)

### 提交预测结果
最后，参赛选手可以将预测结果下载到本地，并上传到[天池](https://tianchi.aliyun.com/competition/entrance/532008/introduction)。